In [7]:
!pip install matplotlib

  Using cached matplotlib-3.3.4-cp36-cp36m-manylinux1_x86_64.whl (11.5 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached Pillow-8.4.0-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached kiwisolver-1.3.1-cp36-cp36m-manylinux1_x86_64.whl (1.1 MB)


In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

uncomment if you need to download the data (you shouldn't need to)

In [9]:
# !wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
# !unzip ml-1m.zip

--2023-05-12 16:11:19--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  27.6MB/s    in 0.2s    

2023-05-12 16:11:19 (27.6 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



set up all of the data (see below)

In [5]:
base_path = './original_dataset/'
users = 'users.dat'
ratings = 'ratings.dat'
movies = 'movies.dat'

ratings_file = base_path + ratings
ratings = pd.read_fwf(ratings_file,header=None)

In [18]:
movies_file = base_path + movies
movies_df = pd.read_fwf(movies_file,header=None)

In [19]:
movies_df

,0,1,2
0,1::Toy Story (1995)::Animation|Children's|Comedy,NaN,NaN
1,2::Jumanji (1995)::Adventure|Children's|Fantasy,NaN,NaN
2,3::Grumpier Old Men (1995)::Comedy|Romance,NaN,NaN
3,4::Waiting to Exhale (1995)::Comedy|Drama,NaN,NaN
4,5::Father of the Bride Part II (1995)::Comedy,NaN,NaN
...,...,...,...
3878,3948::Meet the Parents (2000)::Comedy,NaN,NaN
3879,3949::Requiem for a Dream (2000)::Drama,NaN,NaN
3880,3950::Tigerland (2000)::Drama,NaN,NaN
3881,3951::Two Family House (2000)::Drama,NaN,NaN


In [24]:
movies_df = movies_df[0].str.split('::',expand=True)
movies_df.columns=['MovieID','Name','Genre']

In [25]:
movies_df

,MovieID,Name,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [36]:
movies_df[movies_df['Name'].str.contains('192')]

,MovieID,Name,Genre
945,957,"Scarlet Letter, The (1926)",Drama
1327,1348,"Nosferatu (Nosferatu, eine Symphonie des Graue...",Horror
1856,1925,Wings (1927),Drama|Romance|War
1857,1926,"Broadway Melody, The (1929)",Musical
1941,2010,Metropolis (1926),Sci-Fi
2151,2220,"Manxman, The (1929)",Drama
2152,2221,Blackmail (1929),Thriller
2153,2222,Champagne (1928),Comedy
2154,2223,"Farmer's Wife, The (1928)",Comedy
2155,2224,Downhill (1927),Drama


In [3]:
ratings = ratings[0].str.split('::',expand=True)
ratings.columns=['UserID','MovieID','Rating','Timestamp']

format the data into integer encodings

In [4]:
user_ids = ratings["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

In [5]:
movie_ids = ratings["MovieID"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
ratings["user"] = ratings["UserID"].map(user2user_encoded)
ratings["movie"] = ratings["MovieID"].map(movie2movie_encoded)

In [6]:
num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
ratings["Rating"] = ratings["Rating"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings later
min_rating = min(ratings["Rating"])
max_rating = max(ratings["Rating"])

print(
    "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_movies, min_rating, max_rating
    )
)

Number of users: 6040, Number of Movies: 3706, Min rating: 1.0, Max rating: 5.0


In [16]:
ratings

,UserID,MovieID,Rating,Timestamp,user,movie
0,1,1193,5.0,978300760,0,0
1,1,661,3.0,978302109,0,1
2,1,914,3.0,978301968,0,2
3,1,3408,4.0,978300275,0,3
4,1,2355,5.0,978824291,0,4
...,...,...,...,...,...,...
1000204,6040,1091,1.0,956716,6039,772
1000205,6040,1094,5.0,956704,6039,1106
1000206,6040,562,5.0,9567047,6039,365
1000207,6040,1096,4.0,956715,6039,152


In [63]:
ratings[ratings['MovieID'].astype('int')>=3706]

,UserID,MovieID,Rating,Timestamp,user,movie
138,2,3735,3.0,978298814,1,135
141,2,3809,3.0,978299712,1,138
180,2,3893,1.0,978299535,1,173
191,3,3868,3.0,978298486,2,182
276,5,3793,2.0,978243170,4,237
...,...,...,...,...,...,...
996777,6016,3769,3.0,995664,6015,1477
996791,6016,3783,4.0,963610,6015,2214
999943,6040,3819,5.0,963272,6039,2489
1000183,6040,3735,4.0,960971,6039,135


In [65]:
new_ratings = ratings.groupby('user')['movie'].apply(list).reset_index(name='interactions')

In [66]:
new_ratings['user'] = new_ratings['user'].astype(int)
new_ratings = new_ratings.sort_values(by=['user'])

In [67]:
new_ratings = new_ratings.reset_index(drop=True)

In [68]:
new_ratings

,user,interactions
0,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,1,"[53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 6..."
2,2,"[175, 176, 58, 177, 178, 179, 180, 64, 181, 18..."
3,3,"[63, 64, 208, 209, 210, 44, 48, 113, 124, 127,..."
4,4,"[218, 219, 220, 221, 222, 223, 224, 225, 59, 2..."
...,...,...
6035,6035,"[1892, 2470, 1326, 558, 240, 33, 97, 450, 876,..."
6036,6036,"[97, 256, 1118, 1120, 1492, 128, 2072, 1253, 6..."
6037,6037,"[1615, 153, 1509, 1033, 206, 535, 1528, 1340, ..."
6038,6038,"[33, 115, 2141, 1494, 1495, 608, 1045, 612, 76..."


In [74]:
with open('./train_new.txt', 'w') as f:
#     dfAsString = new_ratings.to_string(header=False, index=False)
#     f.write(dfAsString)
    for index, row in new_ratings.iterrows():
        str_towrite = str(row['user']) + " " + " ".join(map(str, row['interactions'])) + "\n"
        f.write(str_towrite)

In [75]:
with open('./train_new.txt') as f:
    print(f.readline())

0 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52



In [9]:
!pwd

/fs/class-projects/spring2023/cmsc828a/cmsc828ag03/unlearning/data/ml-1m


In [114]:
df = ratings.sample(frac=1, random_state=42)
x = df[["user", "movie"]].values
# Normalize the targets between 0 and 1. Makes it easy to train.
y = df["Rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
# Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * df.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

In [151]:
EMBEDDING_SIZE = 64

class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super().__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)


model = RecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
model.compile(
    loss='mse',
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
)

In [152]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
)

Train on 900188 samples, validate on 100021 samples
Epoch 1/10
900188/900188 [==============================] - 37s 42us/sample - loss: 0.0792 - val_loss: 0.0697
Epoch 2/10
900188/900188 [==============================] - 35s 39us/sample - loss: 0.0667 - val_loss: 0.0642
Epoch 3/10
900188/900188 [==============================] - 35s 39us/sample - loss: 0.0620 - val_loss: 0.0606
Epoch 4/10
900188/900188 [==============================] - 35s 39us/sample - loss: 0.0590 - val_loss: 0.0582
Epoch 5/10
900188/900188 [==============================] - 35s 39us/sample - loss: 0.0570 - val_loss: 0.0567
Epoch 6/10
900188/900188 [==============================] - 35s 39us/sample - loss: 0.0556 - val_loss: 0.0556
Epoch 7/10
900188/900188 [==============================] - 35s 39us/sample - loss: 0.0546 - val_loss: 0.0548
Epoch 8/10
900188/900188 [==============================] - 35s 39us/sample - loss: 0.0539 - val_loss: 0.0542
Epoch 9/10
900188/900188 [==============================] - 35s 39us

# Old Model

In [170]:
embedding_size = 64

In [171]:
user_list = np.arange(num_users)
movie_list = np.arange(num_movies)

In [172]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(num_users, embedding_size, embeddings_initializer="he_normal",
    embeddings_regularizer=keras.regularizers.l2(1e-6),))

model.compile('rmsprop', 'mse')
user_output_array = model.predict(user_list)

In [176]:
minimal_user = user_output_array.squeeze()
minimal_user.shape

(6040, 64)

In [178]:
movie_model = tf.keras.Sequential()
movie_model.add(tf.keras.layers.Embedding(num_movies, embedding_size, embeddings_initializer="he_normal",
    embeddings_regularizer=keras.regularizers.l2(1e-6),))

movie_model.compile('rmsprop', 'mse')
movie_output_array = movie_model.predict(movie_list)

In [179]:
minimal_movie = movie_output_array.squeeze()
minimal_movie.shape

(3706, 64)

In [184]:
minimal_user[0]

array([ 0.02940884,  0.03422559, -0.02134118,  0.0135712 ,  0.00277449,
        0.01527103,  0.0024702 ,  0.01047399,  0.01307029, -0.00638275,
        0.01265904, -0.00623874, -0.02554731, -0.03714249,  0.01871829,
        0.02013095, -0.01522592, -0.03339009, -0.02473376, -0.03465815,
        0.00748808, -0.0035138 , -0.00584937, -0.01653887, -0.00719542,
       -0.00151722,  0.02337624,  0.00081131,  0.01311703, -0.02097753,
        0.01862395,  0.0142702 ,  0.00359323,  0.00803739,  0.00741598,
       -0.02037434, -0.00715928, -0.02026884, -0.00498197, -0.02088491,
       -0.02812333,  0.01179678, -0.01316567,  0.00939347,  0.0378888 ,
        0.00449213,  0.01627946,  0.00604723,  0.0196329 ,  0.01232561,
        0.02353044,  0.00167643,  0.03566212,  0.00267349, -0.00393355,
       -0.01384767,  0.03520547,  0.01645227, -0.00544719,  0.00981106,
       -0.02918231,  0.01828941,  0.02383144, -0.00322443], dtype=float32)

In [185]:
with open('user_pretrain_new.pk', 'wb') as handle:
    pickle.dump(minimal_user, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [186]:
with open('item_pretrain_new.pk', 'wb') as handle:
    pickle.dump(minimal_movie, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [154]:
import pickle

pretrain = pd.read_pickle('user_pretrain.pk')

In [188]:
new_items = pd.read_pickle('item_pretrain_new.pk')

(3706, 64)

In [183]:
pretrain[0]

array([ 0.00219389, -0.04302615, -0.06007273, -0.00323212,  0.01576119,
       -0.04633715, -0.04180552,  0.07051403,  0.09164011,  0.03981549,
        0.00215771, -0.12604086,  0.08228033,  0.04784781, -0.01362647,
        0.07112201,  0.06364987,  0.05925407, -0.00949974,  0.01726384,
        0.00931124, -0.04134427, -0.08695938, -0.05327449, -0.03543516,
       -0.05281315, -0.0398933 , -0.04848811,  0.01409978,  0.07084697,
        0.03145708, -0.01986688, -0.02033977, -0.09155387,  0.07706776,
        0.00625322, -0.00759618, -0.11464418,  0.02779301,  0.011966  ,
       -0.00575389, -0.09850632, -0.02565098, -0.00807661,  0.02078787,
        0.16786245,  0.02849871, -0.01978358, -0.06682788,  0.11486023,
       -0.05014841,  0.07523371,  0.12543403,  0.07935702, -0.06323341,
       -0.04793463, -0.10299463, -0.10140377, -0.000987  , -0.15843594,
       -0.0328703 , -0.05660152, -0.10009345, -0.07483596], dtype=float32)